In [1]:
import gzip
import pandas as pd
import urllib.request
import requests

In [2]:
t_pmids = pd.read_csv('teeth_pmids.txt', header=None)[0].tolist()
t_pmids

['31995917',
 '18810139',
 '11630834',
 '11628701',
 '10257004',
 '10253337',
 '10249335',
 '7471157',
 '7468959',
 '7468637',
 '7464309',
 '7453561',
 '7451769',
 '7446456',
 '7349792',
 '7348704',
 '7348560',
 '7347798',
 '7347011',
 '7347009',
 '7345955',
 '7345136',
 '7344451',
 '7344450',
 '7344449',
 '7344448',
 '7344446',
 '7343673',
 '7342897',
 '7342794',
 '7342559',
 '7341645',
 '7340259',
 '7338553',
 '7338550',
 '7338332',
 '7337363',
 '7337359',
 '7337357',
 '7337356',
 '7337354',
 '7337021',
 '7337020',
 '7337014',
 '7337013',
 '7335645',
 '7333876',
 '7331901',
 '7331761',
 '7331160',
 '7330686',
 '7330685',
 '7329980',
 '7327219',
 '7324616',
 '7319423',
 '7319421',
 '7318737',
 '7314803',
 '7308607',
 '7307062',
 '7306381',
 '7302315',
 '7301853',
 '7299519',
 '7298385',
 '7293722',
 '7293554',
 '7291954',
 '7289873',
 '7289863',
 '7289062',
 '7279090',
 '7274862',
 '7274832',
 '7274527',
 '7270296',
 '7270295',
 '7270294',
 '7270293',
 '7270292',
 '7270291',
 '7270290

In [25]:
pmids=t_pmids
pmids = [pmid for pmid in pmids if pmid.isdigit()]
len(pmids)

345938

In [4]:
non_numeric_pmids = [pmid for pmid in pmids if not pmid.isdigit()]
if non_numeric_pmids:
    print("There are non-numeric PMIDs:", non_numeric_pmids)
else:
    print("All PMIDs are numeric.")


All PMIDs are numeric.


worked, solution was to provide json 

functional code except some minor errors

In [6]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time

def fetch_data(pmids):
    base_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator"
    # Construct the request payload
    payload = {'pmids': pmids, 'concepts': 'Bioconcepts'}
    try:
        response = requests.post(base_url, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        if response.status_code == 200:
            # Split response into individual publications
            publications = response.text.split('\n\n')
            results = []
            for pub_data in publications:
                pub_lines = pub_data.split('\n')
                if len(pub_lines) >= 2:
                    pmid = pub_lines[0].split('|')[0].strip()
                    title = pub_lines[0].split('|')[2].strip()
                    abstract = pub_lines[1].split('|')[2].strip()
                    results.append({'PubMed ID': pmid, 'Title': title, 'Abstract': abstract})
                else:
                    print("No data found for PMID:", pmid)
            return results
        else:
            print("Error fetching data for PMIDs:", pmids)
    except requests.exceptions.RequestException as e:
        print(f"Request failed for PMIDs {pmids}: {e}")
    return []  # Return an empty list if no results are found

# Define batch size
batch_size = 1000

# Split pmids into batches
pmids_batches = [pmids[i:i+batch_size] for i in range(0, len(pmids), batch_size)]

# Fetch data in parallel with tqdm progress tracking
results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    for pmids_batch in tqdm(pmids_batches):
        batch_results = list(executor.map(fetch_data, [pmids_batch]))
        for result_list in batch_results:
            results.extend(result_list)
        time.sleep(0.35)  # Add a small delay between requests to avoid overwhelming the server

# Create DataFrame from results
df_results = pd.DataFrame(results)

# Print the DataFrame
print(df_results)


  0%|          | 0/346 [00:00<?, ?it/s]

No data found for PMID: 31995917


  0%|          | 1/346 [00:01<07:12,  1.25s/it]

No data found for PMID: 30170475


  1%|          | 2/346 [00:02<07:10,  1.25s/it]

...

No data found for PMID: 38293214


100%|█████████▉| 345/346 [08:22<00:01,  1.32s/it]

No data found for PMID: 38293571


100%|██████████| 346/346 [08:23<00:00,  1.46s/it]


       PubMed ID                                              Title  \
0        6108426                    Dental caries and malnutrition.   
1        6110572  Separation of inorganic pyrophosphatase from a...   
2        6112979                             The knocked out tooth.   
3        6113713  Are biomedical exchange programs inside The Pe...   
4        6114139                           Pediatric dental trauma.   
...          ...                                                ...   
342097  38289637  The Rewarding Experience of Serving as a US Ai...   
342098  38290211  Primary apical periodontitis correlates to ele...   
342099  38290450  Curative Effect of Bracketless and Invisible O...   
342100  38293570  Evaluation of cytotoxicity of 3.8 % SDF and Bi...   
342101  38293571  Evaluating smile aesthetic satisfaction and re...   

                                                 Abstract  
0                                                          
1                          

In [24]:
df_results

,PubMed ID,Title,Abstract
0,6108426,Dental caries and malnutrition.,
1,6110572,Separation of inorganic pyrophosphatase from a...,
2,6112979,The knocked out tooth.,
3,6113713,Are biomedical exchange programs inside The Pe...,Between November 1979 and April 1980 we lived ...
4,6114139,Pediatric dental trauma.,
...,...,...,...
342097,38289637,The Rewarding Experience of Serving as a US Ai...,Providing care to service members and veterans...
342098,38290211,Primary apical periodontitis correlates to ele...,AIM: To explore associations between root fill...
342099,38290450,Curative Effect of Bracketless and Invisible O...,Objective: To observe the therapeutic effects ...
342100,38293570,Evaluation of cytotoxicity of 3.8 % SDF and Bi...,Introduction: Nonsurgical endodontic therapies...


In [11]:
df_results.to_csv('title_abstract.tsv', sep='\t', index=False)


In [32]:
from collections import Counter
duplicates = [item for item, count in Counter(pmids).items() if count > 1]
num_duplicates = len(duplicates)

duplicates, num_duplicates


(['31995917',
  '18810139',
  '11630834',
  '11628701',
  '10257004',
  '10253337',
  '10249335',
  '7471157',
  '7468959',
  '7468637',
  '7464309',
  '7453561',
  '7451769',
  '7446456',
  '7349792',
  '7348704',
  '7348560',
  '7347798',
  '7347011',
  '7347009',
  '7345955',
  '7345136',
  '7344451',
  '7344450',
  '7344449',
  '7344448',
  '7344446',
  '7343673',
  '7342897',
  '7342794',
  '7342559',
  '7341645',
  '7340259',
  '7338553',
  '7338550',
  '7338332',
  '7337363',
  '7337359',
  '7337357',
  '7337356',
  '7337354',
  '7337021',
  '7337020',
  '7337014',
  '7337013',
  '7335645',
  '7333876',
  '7331901',
  '7331761',
  '7331160',
  '7330686',
  '7330685',
  '7329980',
  '7327219',
  '7324616',
  '7319423',
  '7319421',
  '7318737',
  '7314803',
  '7308607',
  '7307062',
  '7306381',
  '7302315',
  '7301853',
  '7299519',
  '7298385',
  '7293722',
  '7293554',
  '7291954',
  '7289873',
  '7289863',
  '7289062',
  '7279090',
  '7274862',
  '7274832',
  '7274527',
  '72

In [30]:
duplicates = df_results[df_results.duplicated(subset='PubMed ID')]
if duplicates.empty:
    print("No duplicates found in the 'PubMed ID' column.")
else:
    print("Duplicates found in the 'PubMed ID' column:")
    print(duplicates)


Duplicates found in the 'PubMed ID' column:
       PubMed ID                                              Title  \
7932     7331761  Effects of rehabilitation on the jaws and teet...   
7933     7331901  Plasma exchanges in the treatment of rapidly p...   
7934     7333876  A role for dentistry in managing the oro-nasop...   
7936     7335645  Relation of attrition and mobility of the teet...   
7937     7337013                  [Endodontics of temporary teeth].   
...          ...                                                ...   
342097  38289637  The Rewarding Experience of Serving as a US Ai...   
342098  38290211  Primary apical periodontitis correlates to ele...   
342099  38290450  Curative Effect of Bracketless and Invisible O...   
342100  38293570  Evaluation of cytotoxicity of 3.8 % SDF and Bi...   
342101  38293571  Evaluating smile aesthetic satisfaction and re...   

                                                 Abstract  
7932    The growth of the jaws and teeth wa

In [29]:
pmids_found= df_results['PubMed ID'].tolist()
print("total articles abstract and/or title found with pubtator api :",len(pmids_found))
print(len(set(pmids_found)))

total articles abstract and/or title found with pubtator api : 342102
304110


In [26]:
pmids_found = set(pmids_found)
pmids_set = set(pmids)
pmids_notfound = [pmid for pmid in pmids_set if pmid not in pmids_found]
len(pmids_notfound)

2985

In [27]:
pmids_found = set(df_results['PubMed ID'])
pmids_notfound = set(pmids) - pmids_found
print("Total articles abstract and/or title found with PubTator API:", len(pmids_found))
print("Total PMIDs not found:", len(pmids_notfound))


Total articles abstract and/or title found with PubTator API: 304110
Total PMIDs not found: 2985


['38303353',
 '38443243',
 '38442009',
 '38419993',
 '38348874',
 '38383363',
 '38322815',
 '38434507',
 '38350932',
 '38411484',
 '38426921',
 '38346366',
 '38340133',
 '38376436',
 '38344646',
 '38468123',
 '38338217',
 '38474068',
 '38361137',
 '38378857',
 '38340037',
 '38416293',
 '38321585',
 '38326415',
 '38459266',
 '38380020',
 '38001110',
 '38443613',
 '38368466',
 '38415210',
 '38362555',
 '38304105',
 '32672908',
 '38425600',
 '38360594',
 '38374614',
 '38462493',
 '38367827',
 '38333499',
 '38319938',
 '38419062',
 '38460140',
 '38002290',
 '38352811',
 '38383601',
 '38455178',
 '38363703',
 '38325293',
 '38451936',
 '38417862',
 '31613465',
 '38350985',
 '38457808',
 '38397645',
 '38463476',
 '38475274',
 '38476497',
 '38393294',
 '38389137',
 '38305439',
 '38355115',
 '32491459',
 '37616435',
 '38388598',
 '38462707',
 '38463720',
 '38430349',
 '38448117',
 '38348736',
 '38303737',
 '32310572',
 '34424640',
 '38350113',
 '38317872',
 '38404561',
 '38375392',
 '38408432',